In [ ]:
%run "Setup_DesignStart.ipynb"

In [ ]:
print(target.get_target_name())
print(target.get_fw_buildtime())
print(target.get_fpga_buildtime())

In [ ]:
target.fpga_write(target.REG_SOFT_TRIG_ENABLE, [1])
target.fpga_write(target.REG_COUNTER_QUICK_START, [1])
target.fpga_write(target.REG_CAPTURE_RAW, [1])
target.fpga_write(target.REG_COUNT_WRITES, [0])
target.fpga_write(target.REG_SOFT_TRIG_PASSTHRU, [1])

In [ ]:
# match on any PC trace packet:
target.set_pattern_match(0, [5, 8, 32, 0, 0, 0, 0, 0], [255, 255, 255, 0, 0, 0, 0, 0])
#target.set_pattern_match(0, [5, 8, 32, 229, 2, 0, 0, 132], [255, 255, 255, 255, 255, 255, 255, 0])

# enable matching rule:
target.fpga_write(target.REG_PATTERN_ENABLE, [1])

target.fpga_write(target.REG_CAPTURE_LEN, int.to_bytes(10000, length=4, byteorder='little'))

In [ ]:
target.set_reg('DWT_COMP0', '000012a4')
target.set_reg('DWT_COMP1', '000012fc')

In [ ]:
target.set_reg('ETM_TEEVR', '000150a0') # COMP0 or COMP1
#target.set_reg('ETM_TEEVR', '00000020') # COMP0 only
#target.set_reg('ETM_TEEVR', '00000021') # COMP1 only

In [ ]:
target.arm_trace()

In [ ]:
target.synced()

In [ ]:
target.simpleserial_write('p', bytearray(16), printresult=True)

In [ ]:
#Capture Traces
from tqdm import tnrange
import numpy as np
import time

num_traces = 5
ktp = cw.ktp.Basic()

traces = []

for i in tnrange(num_traces, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    trace = cw.capture_trace(scope, target.ss, text, key)

    if trace is None:
        continue
    traces.append(trace)

#Convert traces to numpy arrays
trace_array = np.asarray([trace.wave for trace in traces])  # if you prefer to work with numpy array for number crunching
textin_array = np.asarray([trace.textin for trace in traces])
known_keys = np.asarray([trace.key for trace in traces])  # for fixed key, these keys are all the same

In [ ]:
target.fpga_read(target.REG_TRACE_COUNT, 8)

In [ ]:
target.fifo_empty()

In [ ]:
raw = target.read_capture_data()

In [ ]:
len(raw)

In [ ]:
#target.print_raw_data(raw)

In [ ]:
frames = target.get_raw_trace_packets(raw, True)

In [ ]:
len(frames)

In [ ]:
allframedata = []
for r in raw:
    command = r[2] & 0x3
    if command == target.FE_FIFO_CMD_STAT:
        data = r[1]
        allframedata.append(data)

In [ ]:
allframedata